In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
import json


In [ ]:
def cor_predominante(imagem):
    """Detecta cor predominante (média RGB)"""
    media = imagem.mean(axis=(0,1))
    return tuple(int(v) for v in media)

def carregar_imagem(caminho):
    if not os.path.exists(caminho):
        print("Erro: Caminho inválido.")
        return None
    img = cv2.imread(caminho)
    if img is None:
        print("Erro ao carregar imagem.")
        return None
    print(f"Imagem carregada com sucesso: {caminho}")
    print(f"Dimensões: {img.shape[1]}x{img.shape[0]} px")
    print("Cor predominante (RGB):", cor_predominante(img))
    return img


In [ ]:
def main():
    print("=== SISTEMA DE ANÁLISE DE PADRÕES EM IMAGENS ===")
    caminho = input("Digite o caminho da imagem (ex: /content/imagem.jpg): ")
    img = carregar_imagem(caminho)
    if img is None:
        return

    while True:
        print("\n--- MENU ---")
        print("1. Converter para escala de cinza")
        print("2. Converter para HSV")
        print("3. Equalizar histograma")
        print("4. Binarizar imagem")
        print("5. Detectar bordas")
        print("6. Calcular métricas de textura")
        print("7. Exibir mapa de textura")
        print("8. Gerar relatório manual")
        print("9. Detectar formas geométricas")
        print("0. Sair")

